# Session 1.2: Handling your Coordinates

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Author: Dr Richard Gowers   
Email: richardjgowers@gmail.com

## Learning outcomes:
* How to access atom positions
* Calculating pairwise distances
* Calculating angles
* How box information is stored and how this relates to periodic boundaries.
* Efficient distance calculations and sparse arrays
* Manipulating `Universe.trajectory` to select a frame

**Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

## Loading the system

We will be working with the same system as the previous session:

In [1]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD

u = mda.Universe(PSF, DCD)

print(u)

<Universe with 3341 atoms>


# 4. Accessing coordinates

The most important attribute of your atoms is undoubtedly their positions! The position information is made available via an `AtomGroup` in the `positions` attribute:

In [ ]:
Ca = u.select_atoms('type CA')

Ca.positions

This returns a `numpy` array, which can be easily manipulated. For example, we can could calulate the center of geometry by:

In [2]:
import numpy as np
ca = u.select_atoms('name CA')
cog = np.mean(ca.positions, axis=0)
print(cog)

[ 0.06873626 -0.04605844 -0.246437  ]


Alternatively, we can use the built-in method:

In [3]:
ca.center_of_geometry()

array([ 0.06873595, -0.04605918, -0.24643682])

Other convenient methods for common calculations based on positions include `center_of_mass()`, `radius_of_gyration()` and `principal_axes()`.

### Exercise 4


Let's look in more detail at the AdK protein. AdK has three domains:

 - CORE (residues 1-29, 60-121, 160-214)
 - NMP (residues 30-59)
 - LID (residues 122-159)


<center><img src="imgs/adk.png" alt="mda" style="width: 300px;"/></center>

**4a. Calculate the center of mass of each of the three domains**

In [4]:
domains = {
           'CORE': u.select_atoms("resid 1-29 60-121 160-214"),
           'NMP': u.select_atoms("resid 30-59"),
           'LID': u.select_atoms("resid 122-159")
          }
cogs = {name: ag.center_of_mass() for name,ag in domains.items()} 
print(cogs)

{'CORE': array([4.564116  , 2.08700105, 1.54992649]), 'NMP': array([ -3.20330174, -13.60247613,  -3.06221538]), 'LID': array([-15.11337499,   2.12292226,  -4.40910485])}


How do these differ from the centers of geometry?

## Calculating angles

Angles between these domains can be used to distinguish open and closed states of the protein (see the figure above) [1]. These angles are defined between the center of geometry of the backbone and C$_\beta$ atoms of the following groups of atoms:

 - $\theta_{NMP}$ is defined between residues:
   - A: 115-125 
   - B: 90-100
   - C: 35-55
 - $\theta_{LIC}$ is defined between residues:
   - A: 179-185
   - B: 112-125
   - C: 125-153 
 
**4b. Calculate the current values of these angles**

Hints:
 - The angle between two vectors is given by:
$$
\theta = arccos\left( \frac{\vec{BA}\cdot\vec{BC}}{|\vec{BA}||\vec{BC}|} \right)
$$

- You can use numpy `numpy.linalg.norm()` to calculate the norm of a vector. Numpy also has functions `numpy.arccos()` and `numpy.dot()`.


In [5]:
import numpy as np
from numpy.linalg import norm

A_NMP = u.select_atoms('resid 115-125 and (backbone or name CB)').center_of_geometry()
B_NMP = u.select_atoms('resid 90-100 and (backbone or name CB)').center_of_geometry()
C_NMP = u.select_atoms('resid 35-55 and (backbone or name CB)').center_of_geometry()
BA_NMP = A_NMP - B_NMP
BC_NMP = C_NMP - B_NMP
theta_NMP = np.arccos(np.dot(BA_NMP, BC_NMP)/(norm(BA_NMP)*norm(BC_NMP)))
print('theta_NMP: ', np.rad2deg(theta_NMP))


A_LID = u.select_atoms('resid 179-185 and (backbone or name CB)').center_of_geometry()
B_LID = u.select_atoms('resid 112-125 and (backbone or name CB)').center_of_geometry()
C_LID = u.select_atoms('resid 125-153 and (backbone or name CB)').center_of_geometry()
BA_LID = A_LID - B_LID
BC_LID = C_LID - B_LID
theta_LID = np.arccos(np.dot(BA_LID, BC_LID)/(norm(BA_LID)*norm(BC_LID)))
print('theta_LID: ', np.rad2deg(theta_LID))

theta_NMP:  44.124820978242475
theta_LID:  107.21578150193177



- Alternatively the function `MDAnalysis.lib.distances.calc_angles` is designed for this purpose!  Importantly this function can handle periodic boundary conditions...

In [12]:
theta_NMP = mda.lib.distances.calc_angles(A_NMP, B_NMP, C_NMP)
theta_LID = mda.lib.distances.calc_angles(A_LID, B_LID, C_LID)

print('theta_NMP: ', np.rad2deg(theta_NMP))
print('theta_LID: ', np.rad2deg(theta_LID))

theta_NMP:  44.12482298624018
theta_LID:  107.21578112990201


## Next Notebook

[Getting started with Python](Session_1.2.ipynb)